<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Generate Leads from likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_comments_from_post.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #linkedin #post #comments #naas_drivers

## Input

### Import libraries

In [1]:
from naas_drivers import linkedin, hubspot
import pandas as pd
import numpy as np
import naas
from datetime import datetime, timedelta
import requests
import json

### Setup your LinkedIn
- Get your cookies : <a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [2]:
# Lindekin cookies
LI_AT = "AQEDARCNSioFAHAEAAABfvQsStIAAAF_GDjO0lYAFsSVNvaEPJa5ZpknkX706RXB4OXEY5jryrf1yHvHwqAk54kVmf-tNL3tdGcnTpNrzGXAAMeFTeLTbzjCj2Hj3bO6Go6v66yV38AN9J316NNNKTEN"
JSESSIONID = "ajax:2913276942853423494"

# Enter your profile URL
PROFILE_URL = "https://www.linkedin.com/in/florent-ravenel/"

### Setup your Hubspot
- Get your API token
- Get your hubspot owner ID
- Get your hubspot company ID

In [3]:
#initial parameters for hubspot connection
HS_CONTACT_ASSIGN_TO_ID = "41252017" #contact id to whom task needs to be assigned
HS_API_TOKEN = "7696655f-3aff-4524-84f1-5c988abdab9f"
HUB_COMPANY_ID = "2474088"

In [ ]:
### 

In [ ]:
SEND_EMAIL_TO = ""

## Model

### Get posts from feed

In [4]:
df_posts = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(PROFILE_URL, count=100, limit=-1)
df_posts

,ACTIVITY_ID,PAGINATION_TOKEN,PUBLISHED_DATE,AUTHOR_NAME,SUBDESCRIPTION,TITLE,TEXT,CHARACTER_COUNT,TAGS,TAGS_COUNT,...,IMAGE_URL,POLL_ID,POLL_QUESTION,POLL_RESULTS,POST_URL,VIEWS,COMMENTS,LIKES,SHARES,ENGAGEMENT_SCORE
0,6762685443919101952,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,1 year ago,"Quand je parle de mon travail à mes proches, i...","Quand je parle de mon travail à mes proches, i...",715,#netflix #jupyternotebook #data #python,4,...,https://media-exp1.licdn.com/dms/image/C4D22AQ...,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,2440,2,19,0,0.0086
1,6760161338296676352,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,1 year ago,Do it yourself ! Automatisez le suivi de votre...,Do it yourself ! Automatisez le suivi de votre...,588,#qonto #api #python,3,...,https://media-exp1.licdn.com/dms/image/sync/C4...,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,1576,4,24,0,0.0178
2,6757251421881548800,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,1 year ago,Do it yourself ! Pour suivre ses comptes banca...,Do it yourself ! Pour suivre ses comptes banca...,766,#design #excel #python #datascience #training,5,...,https://media-exp1.licdn.com/dms/image/sync/C4...,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,2891,12,43,0,0.0190
3,6752541502989197312,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,1 year ago,Vous en avez marre d’aller exporter vos relevé...,Vous en avez marre d’aller exporter vos relevé...,1252,#python #api,2,...,https://media-exp1.licdn.com/dms/image/sync/C5...,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,1979,6,18,0,0.0121
4,6729737309060644864,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,1 year ago,"Dans ma vie professionnelle, je n'ai jamais ai...","Dans ma vie professionnelle, je n'ai jamais ai...",1069,None,0,...,https://media-exp1.licdn.com/dms/image/sync/C5...,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,6826,24,77,0,0.0148
5,6656913392814698496,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,1 year ago,World Situation Room : Après un mois de confin...,World Situation Room : Après un mois de confin...,894,#restezchezvous #stayathome,2,...,https://media-exp1.licdn.com/dms/image/C5622AQ...,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,4484,6,46,0,0.0116
6,6650397549905739776,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,2 years ago,"COVID-19, Economie : Des chiffres dans tous le...","COVID-19, Economie : Des chiffres dans tous le...",930,#restezchezvous #stayathome,2,...,https://media-exp1.licdn.com/dms/image/C5622AQ...,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,4914,6,83,0,0.0181
7,6187602452842315776,dXJuOmxpOmFjdGl2aXR5OjU4MDcxNDA2OTcyMjg2MDMzOT...,2013-11-15 15:44:56+01:00,Florent Ravenel,5 years ago,None,None,0,[],0,...,None,None,None,None,https://www.linkedin.com/feed/update/urn:li:ac...,0,0,1,0,0.0000


### Get likes from posts

In [5]:
def get_likes(df):
    df_likes = pd.DataFrame()
    for index, row in df.iterrows():
        post_url = row['POST_URL']
        tmp_df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(post_url)
        if not tmp_df.empty:
            df_likes = pd.concat([df_likes, tmp_df])
    return df_likes
        
df_likes = get_likes(df_posts)
df_likes

,PROFILE_URN,PROFILE_ID,FIRSTNAME,LASTNAME,OCCUPATION,REACTION_TYPE,POST_URL
0,ACoAABUb6UsBjoejmZICw4lKY-dkou9qyeMHpJ8,goran-stefanovic-40b7309a,Goran,Stefanovic,Build trust with privacy @Didomi,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...
1,ACoAABYdDD8Bmz1xbob0K071XWBDIkeC2CWIuhE,giulia-penisson,Giulia,Penisson,Consultante,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...
2,ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg,jérémy-ravenel-8a396910,Jérémy,Ravenel,⚡️ Building @naas.ai - open source data platfo...,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...
3,ACoAABX-HMgBeQ0R9TECFDwMYIOu7QJE2JZ4Kcs,romainhernot,Romain,HERNOT,Senior Associate - Direct Lending at H.I.G. Wh...,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...
4,ACoAABRgzzwBMeaHrEbVeSFMOdubipY129NkxlU,juliedavidmarketing,Julie,David,Communication Officer Klaxoon,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...
...,...,...,...,...,...,...,...
78,ACoAACSkZuoBakmcBNT96QHb28ES79cbz0jXj8c,gabriel-wargnier-digital-marketing-seo,Gabriel,Wargnier,Digital marketing and SEO specialist,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...
79,ACoAAAdKbJEBC3hqNUOyiPuHyeaMfd2JCSKhvIg,gwenn-charlot-a41b6934,Gwenn,Charlot,Head of Growth Acquisition @ Qonto,PRAISE,https://www.linkedin.com/feed/update/urn:li:ac...
80,ACoAAAqUo80BtL3Jt9jtdJ2YmujixTlx-kFeEqM,yannis-himmich-8115474b,Yannis,Himmich,"Director, Strategic Accounts",LIKE,https://www.linkedin.com/feed/update/urn:li:ac...
81,ACoAAAI0hz4BmOQIYYutD8ebsL4bTtUtolZAqZ0,alexandreparlange,Alexandre,Parlange,Entrepreneur at Antler,LIKE,https://www.linkedin.com/feed/update/urn:li:ac...


In [6]:
df_likes.POST_URL.unique()

array(['https://www.linkedin.com/feed/update/urn:li:activity:6762685443919101952',
       'https://www.linkedin.com/feed/update/urn:li:activity:6760161338296676352',
       'https://www.linkedin.com/feed/update/urn:li:activity:6757251421881548800',
       'https://www.linkedin.com/feed/update/urn:li:activity:6752541502989197312',
       'https://www.linkedin.com/feed/update/urn:li:activity:6729737309060644864',
       'https://www.linkedin.com/feed/update/urn:li:activity:6656913392814698496',
       'https://www.linkedin.com/feed/update/urn:li:activity:6650397549905739776',
       'https://www.linkedin.com/feed/update/urn:li:activity:6187602452842315776'],
      dtype=object)

In [9]:
df_counts = df_likes.copy()

df_counts = df_counts.groupby(["PROFILE_URN",
                               "PROFILE_ID",
                               "FIRSTNAME",
                               "LASTNAME"], as_index=False).agg({"POST_URL": "count"})
df_counts = df_counts.sort_values(by="POST_URL", ascending=False)
df_counts

,PROFILE_URN,PROFILE_ID,FIRSTNAME,LASTNAME,POST_URL
42,ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg,jérémy-ravenel-8a396910,Jérémy,Ravenel,8
140,ACoAABYVsxEBFJ5cAeOCukhSw2dAK8h_pMtwg1E,valentin-piquard-955105a4,Valentin,Piquard,7
56,ACoAAAWiY7UBkYRLS1RWTuAeJ6uyRdqbAYTqUSc,alexis-yvin-b597a727,Alexis,Yvin,5
2,ACoAAA0dw_kBBJYe_2mzp8Yh7d_GpWXHwu_Uq3g,martindonadieu,🚀 Martin,DONADIEU,5
4,ACoAAA0vMdEB2m6Re52ReZ7s4ndgUCIaQY8R9Tg,nicolas-bannier-ducros-4690b362,Nicolas,Bannier Ducros,5
...,...,...,...,...,...
81,ACoAAAs1iw8Bh79pzjqOZ6t9cKX-xpOnCGLi6TQ,laura-imberton-0bbb9252,Laura,Imberton,1
82,ACoAAAsUYskBKTUOhDad6kkkx5wzb-ad61xeuqQ,arimbr,Ari,Bajo Rouvinen,1
83,ACoAAAsfZ04By5ReK44d73hfrikzK0Lf9pdKdKQ,victor-de-la-paillonne-4ba5b252,Victor,de la Paillonne,1
84,ACoAAAtDQWYBsfnoX6fPKeD0BhSUDHpc5eOQ79w,valérie-piquard-17234653,Valérie,Piquard,1


- Grouping the like count and classify the leads as per levels i.e. <b>L1, L2, L3, L4.</b> ( from highest to lowest counts of likes) with column name <b>"POTENTIAL LEAD"</b>
- Group posts as per the the column <b>"POTENTIAL LEAD"</b>

In [11]:
conditions = [
    (df_counts['POST_URL'] <= 2),
    (df_counts['POST_URL'] > 2) & (df_counts['POST_URL'] <= 9),
    (df_counts['POST_URL'] > 9) & (df_counts['POST_URL'] <= 15),
    (df_counts['POST_URL'] > 15)
    ]

values = ['L4', 'L3', 'L2', 'L1']

In [13]:
df_counts['POTENTIAL_LEAD'] = np.select(conditions, values)
df_counts

,PROFILE_URN,PROFILE_ID,FIRSTNAME,LASTNAME,POST_URL,POTENTIAL_LEAD
42,ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg,jérémy-ravenel-8a396910,Jérémy,Ravenel,8,L3
140,ACoAABYVsxEBFJ5cAeOCukhSw2dAK8h_pMtwg1E,valentin-piquard-955105a4,Valentin,Piquard,7,L3
56,ACoAAAWiY7UBkYRLS1RWTuAeJ6uyRdqbAYTqUSc,alexis-yvin-b597a727,Alexis,Yvin,5,L3
2,ACoAAA0dw_kBBJYe_2mzp8Yh7d_GpWXHwu_Uq3g,martindonadieu,🚀 Martin,DONADIEU,5,L3
4,ACoAAA0vMdEB2m6Re52ReZ7s4ndgUCIaQY8R9Tg,nicolas-bannier-ducros-4690b362,Nicolas,Bannier Ducros,5,L3
...,...,...,...,...,...,...
81,ACoAAAs1iw8Bh79pzjqOZ6t9cKX-xpOnCGLi6TQ,laura-imberton-0bbb9252,Laura,Imberton,1,L4
82,ACoAAAsUYskBKTUOhDad6kkkx5wzb-ad61xeuqQ,arimbr,Ari,Bajo Rouvinen,1,L4
83,ACoAAAsfZ04By5ReK44d73hfrikzK0Lf9pdKdKQ,victor-de-la-paillonne-4ba5b252,Victor,de la Paillonne,1,L4
84,ACoAAAtDQWYBsfnoX6fPKeD0BhSUDHpc5eOQ79w,valérie-piquard-17234653,Valérie,Piquard,1,L4


### Get profile details based on the POST_URL above
- filter rows from "grouped_profile_posts" dataframe based on L1, L2, & L3 levels and add data to leads_list
- Extract EMAIL, FIRSTNAME, LASTNAME, PHONE_NUMBER, OCCUPATION from profile URL

In [18]:
leads_list = [];

r_count = 1
for index, row in df_counts.iterrows():
    if row['POTENTIAL_LEAD'] == "L3" or row['POTENTIAL_LEAD'] == "L2" or row['POTENTIAL_LEAD'] == "L1":
        profileid = row['PROFILE_URN']
        profileurl = "https://www.linkedin.com/in/"+profileid+"/"
       
        PROFILECONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(profileurl)
        PROFILEIDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(profileurl)
        
        profileemail = PROFILECONTACTS.at[0,'EMAIL']
        profilephoneno = PROFILECONTACTS.at[0,'PHONENUMBER']
        profilename = PROFILEIDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILEIDENTITY.at[0,'LASTNAME']
        profilefirstname = PROFILEIDENTITY.at[0,'FIRSTNAME']
        profilelastname = PROFILEIDENTITY.at[0,'LASTNAME']
        profileoccupation = PROFILEIDENTITY.at[0,'OCCUPATION']
        if profileemail != None :
            leads_list.append([r_count,profilename,profilefirstname,profilelastname,profileemail,profilephoneno,profileoccupation,profileurl])
            r_count = r_count + 1

### Create hubspot contacts from linkedin likes

In [1]:
#to store the resulting output of create contact method
huboutput = ""

<h4>HS_CONTACT_ID_LIST</h4> - The list to store the contact id recently created and the linkedin profile URL of associated contact

In [20]:
HS_CONTACT_ID_LIST = []
for i in leads_list:
        profilename = i[1]
        profilefirstname = i[2]
        profilelastname = i[3]
        profileemail = i[4]
        profilephoneno = i[5]
        profileoccupation = i[6]
        profileurl = i[7]
        # With send method
        data = {"properties": 
                {
                    "linkedinbio": profileurl,
                    "firstname": profilefirstname,
                    "lastname": profilelastname,
                    "jobtitle": profileoccupation,
                    "email": profileemail,
                    "phone": profilephoneno,
                }
               }
        
        #write data to CRM ( create new contact in CRM)
        huboutput = hubspot.connect(HS_API_TOKEN).contacts.send(data)
        HS_CONTACT_ID_LIST.append([huboutput,profileurl])

✔️ Contact (id=1051) successfully created.
✔️ Contact (id=1101) successfully created.
✔️ Contact (id=1151) successfully created.
✔️ Contact (id=1201) successfully created.
✔️ Contact (id=1251) successfully created.
✔️ Contact (id=1301) successfully created.
✔️ Contact (id=1351) successfully created.
✔️ Contact (id=1401) successfully created.
✔️ Contact (id=1451) successfully created.
✔️ Contact (id=1501) successfully created.


## Output

### Display result - Create tasks for user and send email notification

In [21]:
HUB_CONTACT_URL = "https://app.hubspot.com/contacts/"+HUB_COMPANY_ID+"/contact/"

ENGAGEMENT_URL = "https://api.hubapi.com/engagements/v1/engagements"

QUERY_STRING = { "hapikey": HS_API_TOKEN }

#email parameter(s)


#timestamp delay to set due date for tasks
tstampobj = datetime.now() + timedelta(days=10)
tstamp = tstampobj.timestamp() * 1000

for i in HS_CONTACT_ID_LIST:
    hs = hubspot.connect(HS_API_TOKEN)
    contact_id = i[0]
    contact = hs.contacts.get(contact_id)
    contact_props = contact.get('properties')
    contact_linkedin_url = i[1]
    hs_contact_url = HUB_CONTACT_URL + contact_id
    payload = json.dumps({
        "engagement": {
            "active": 'true',
            "ownerId": "",
            "type": "TASK",
            "timestamp": tstamp
        },
        "associations": {
            "contactIds": [HS_CONTACT_ASSIGN_TO_ID],
            "companyIds": [],
            "dealIds": [],
            "ownerIds": [],
        },

        "metadata": {
            "body": "Hi there, you need to contact following user & task is already assigned to you.<br/>" + "Name :" +contact_props['firstname']+ contact_props['lastname'] + " Contact URL : " + hs_contact_url,
            "subject": "Task created for Contact ID :"+ contact_id,
            "status": "NOT_STARTED",
            "forObjectType": "CONTACT"
        }
    });
    headers = {
        'Content-Type': "application/json",
    }
    response = requests.request("POST", ENGAGEMENT_URL, data=payload, headers=headers, params=QUERY_STRING)

    if response.status_code == 200:
        print("Task created successfully for Contact ID :",contact_id)
        email_to = SEND_EMAIL_TO #to send the report
        subject = "LinkedIN Leads Alert"
        content = f"<p>Hi there,</p><br/><p>A new task has been created for you with contact ID :" + contact_id + "</p><br/><p>Actions : <a href="+hs_contact_url+">Open Hubspot Contact</a> | <a href="+contact_linkedin_url+">Open LinkedIN Profile</a></p>"

        naas.notification.send(email_to=email_to, subject=subject, html=content)
    else:
        print("Could not create the task for Contact ID :"+contact_id+", ask for on hello@naas.ai for more details")

Task created successfully for Contact ID : 1051
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1101
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1151
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1201
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1251
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1301
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1351
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1401
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1451
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 1501
👌 💌 Email has been sent successfully !
